## Manuseando RDDs
Neste notebook iremos aprender a realizar operações em RDDs

### Importando as bibliotecas
Nesta etapa iremos apenas importar todas as bibliotecas e funções necessárias para rodar o programa

In [3]:
from pyspark.sql import SparkSession

### Criando uma SparkSession
Por meio de uma SparkSession terei acesso ao SparkContext da minha aplicação.

In [5]:
spark = SparkSession.builder.appName('Select').getOrCreate()

### Carregando os dados
Nesta etapa estamos carregando os dados que utilizaremos neste notebook

In [7]:
dados = spark.sparkContext.textFile("/FileStore/tables/student_mat.csv")

### Retorna as primeiras linhas do rdd
Note que quando carregamos os dados, criamos um RDD[String], ou seja, cada linha
do nosso arquivo está em uma posicao do RDD

In [9]:
dados.take(10)

Out[12]: ['school;sex;age;address;famsize;Pstatus;Medu;Fedu;Mjob;Fjob;reason;guardian;traveltime;studytime;failures;schoolsup;famsup;paid;activities;nursery;higher;internet;romantic;famrel;freetime;goout;Dalc;Walc;health;absences;G1;G2;G3',
 '"GP";"F";18;"U";"GT3";"A";4;4;"at_home";"teacher";"course";"mother";2;2;0;"yes";"no";"no";"no";"yes";"yes";"no";"no";4;3;4;1;1;3;6;"5";"6";6',
 '"GP";"F";17;"U";"GT3";"T";1;1;"at_home";"other";"course";"father";1;2;0;"no";"yes";"no";"no";"no";"yes";"yes";"no";5;3;3;1;1;3;4;"5";"5";6',
 '"GP";"F";15;"U";"LE3";"T";1;1;"at_home";"other";"other";"mother";1;2;3;"yes";"no";"yes";"no";"yes";"yes";"yes";"no";4;3;2;2;3;3;10;"7";"8";10',
 '"GP";"F";15;"U";"GT3";"T";4;2;"health";"services";"home";"mother";1;3;0;"no";"yes";"yes";"yes";"yes";"yes";"yes";"yes";3;2;2;1;1;5;2;"15";"14";15',
 '"GP";"F";16;"U";"GT3";"T";3;3;"other";"other";"home";"father";1;2;0;"no";"yes";"yes";"no";"yes";"yes";"no";"no";4;3;2;1;2;5;4;"6";"10";10',
 '"GP";"M";16;"U";"LE3";"T";4;3;"services";"other";"reputation";"mother";1;2;0;"no";"yes";"yes";"yes";"yes";"yes";"yes";"no";5;4;2;1;2;5;10;"15";"15";15',
 '"GP";"M";16;"U";"LE3";"T";2;2;"other";"other";"home";"mother";1;2;0;"no";"no";"no";"no";"yes";"yes";"yes";"no";4;4;4;1;1;3;0;"12";"12";11',
 '"GP";"F";17;"U";"GT3";"A";4;4;"other";"teacher";"home";"mother";2;2;0;"yes";"yes";"no";"no";"yes";"yes";"no";"no";4;1;4;1;1;1;6;"6";"5";6',
 '"GP";"M";15;"U";"LE3";"A";3;2;"services";"other";"home";"mother";1;2;0;"no";"yes";"yes";"no";"yes";"yes";"yes";"no";4;2;2;1;1;1;0;"16";"18";19']

### Se quiser pegar acessar a primeira linha como String, podemos fazer da seguinte forma

In [11]:
dados.take(10)[0]

Out[17]: 'school;sex;age;address;famsize;Pstatus;Medu;Fedu;Mjob;Fjob;reason;guardian;traveltime;studytime;failures;schoolsup;famsup;paid;activities;nursery;higher;internet;romantic;famrel;freetime;goout;Dalc;Walc;health;absences;G1;G2;G3'

### Agora vamos retirar o cabeçalho do nosso RDD

In [13]:
header = dados.take(1)[0]
resultados = dados.filter(lambda x : x != header)
resultados.take(1)

Out[18]: ['"GP";"F";18;"U";"GT3";"A";4;4;"at_home";"teacher";"course";"mother";2;2;0;"yes";"no";"no";"no";"yes";"yes";"no";"no";4;3;4;1;1;3;6;"5";"6";6']

### Agora iremos transformar nosso RDD [String]

In [15]:
resultados = dados.map(lambda x : x.split(";"))

### Selecionando colunas específicas
Uma vez que temos um RDD [Array[String]] podemos selecionar uma coluna especifica. Lembrando das posicoes das colunas:
school;sex;age;address;famsize;Pstatus;Medu;Fedu;Mjob;Fjob;reason;guardian;traveltime;studytime;failures;schoolsup;famsup;paid;activities;nursery;higher;internet;romantic;famrel;freetime;goout;Dalc;Walc;health;absences;G1;G2;G3

In [17]:
# Seleciona a coluna school
resultados.map(lambda x : x[0]).take(10)

Out[21]: ['school',
 '"GP"',
 '"GP"',
 '"GP"',
 '"GP"',
 '"GP"',
 '"GP"',
 '"GP"',
 '"GP"',
 '"GP"']

In [18]:
# Seleciona a coluna sex
resultados.map(lambda x : x[1]).take(10)

Out[22]: ['sex', '"F"', '"F"', '"F"', '"F"', '"F"', '"M"', '"M"', '"F"', '"M"']

### Agora iremos colocar todos os comandos juntos para selecionar a coluna idade (já convertendo a idade para Int)

In [20]:
header = dados.take(1)[0]
dados.filter(lambda x : x != header) \
    .map(lambda x : x.split(";")) \
    .map(lambda x : int(x[2])) \
    .take(10)

Out[24]: [18, 17, 15, 15, 16, 16, 16, 17, 15, 15]

### Verifica a quantidade de registros

In [22]:
dados.count()

Out[26]: 396

### Calcula a média da coluna idade manualmente

In [24]:
header = dados.take(1)[0]
contagem = dados.count()
soma = dados.filter(lambda x : x != header).map(lambda x : x.split(";")).map(lambda x : int(x[2])).sum()
media = soma / contagem
media

Out[35]: 16.654040404040405

### Filtra apenas sex = "M"

In [26]:
header = dados.take(1)[0]
dados.filter(lambda x : x != header).map(lambda x : x.split(";")).filter(lambda x : x[1] == '"M"').count()

Out[42]: 187

### Seleciona as três primeiras colunas

In [28]:
resultados = dados.filter(lambda x : x != header).map(lambda x : x.split(";")).map(lambda x : [x[0], x[1], x[2]])
resultados.take(10)

Out[45]: [['"GP"', '"F"', '18'],
 ['"GP"', '"F"', '17'],
 ['"GP"', '"F"', '15'],
 ['"GP"', '"F"', '15'],
 ['"GP"', '"F"', '16'],
 ['"GP"', '"M"', '16'],
 ['"GP"', '"M"', '16'],
 ['"GP"', '"F"', '17'],
 ['"GP"', '"M"', '15'],
 ['"GP"', '"M"', '15']]

### Obrigado!
Quer construir uma carreira em Data Science? Acesse meu blog pessoal em https://www.hackinganalytics.com/